In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,366 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,047 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal 

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# Show schema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [5]:
# Import date time functions
from pyspark.sql.functions import year

# Show the year for the date column
df.select(year(df["date"])).show()

+----------+
|year(date)|
+----------+
|      2022|
|      2021|
|      2019|
|      2019|
|      2022|
|      2019|
|      2020|
|      2019|
|      2020|
|      2021|
|      2019|
|      2021|
|      2020|
|      2021|
|      2019|
|      2021|
|      2021|
|      2019|
|      2019|
|      2019|
+----------+
only showing top 20 rows



In [6]:
# Save the year as a new column
df = df.withColumn("year", year(df['date']))
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|2019-01-30|      2

In [7]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [11]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query_1 = '''
SELECT year, round(avg(price),2) as `average price for 4bdrm house`
FROM home_sales
WHERE bedrooms = 4
GROUP BY year
'''
spark.sql(query_1).show()

+----+-----------------------------+
|year|average price for 4bdrm house|
+----+-----------------------------+
|2022|                    296363.88|
|2019|                     300263.7|
|2020|                    298353.78|
|2021|                    301819.44|
+----+-----------------------------+



In [12]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query_2 = '''
SELECT year, round(avg(price),2) as `average price for 3bdrm, 3bthrm`
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year
'''
spark.sql(query_2).show()


+----+-------------------------------+
|year|average price for 3bdrm, 3bthrm|
+----+-------------------------------+
|2022|                      292725.69|
|2019|                      287287.82|
|2020|                      294204.16|
|2021|                      294211.46|
+----+-------------------------------+



In [13]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query_3= '''
SELECT year, round(avg(price),2) as `average price for 3bdrm, 3 bthrm, w 2 flrs, and greater than or equal to 2,000 sqft `
FROM home_sales
WHERE bedrooms = 3 
AND bathrooms = 3
AND floors = 2
AND sqft_lot >= 2000
GROUP BY year
'''
spark.sql(query_3).show()


+----+------------------------------------------------------------------------------------+
|year|average price for 3bdrm, 3 bthrm, w 2 flrs, and greater than or equal to 2,000 sqft |
+----+------------------------------------------------------------------------------------+
|2022|                                                                           287162.18|
|2019|                                                                           282752.72|
|2020|                                                                           293176.41|
|2021|                                                                           290319.06|
+----+------------------------------------------------------------------------------------+



In [17]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query_4 ='''
SELECT view, round(avg(price),2) as `view rating for the average price of a home where homes are greater than or equal to $350,000`
FROM home_sales
WHERE price >= 350000 
GROUP BY view
ORDER BY view desc
'''
spark.sql(query_4).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------------------------------------------------------------------------------------------+
|view|view rating for the average price of a home where homes are greater than or equal to $350,000|
+----+---------------------------------------------------------------------------------------------+
|  99|                                                                                   1061201.42|
|  98|                                                                                   1053739.33|
|  97|                                                                                   1129040.15|
|  96|                                                                                   1017815.92|
|  95|                                                                                    1054325.6|
|  94|                                                                                    1033536.2|
|  93|                                                                                   10

In [19]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [20]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [21]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query_cached = '''
SELECT view, round(avg(price),2) as `view rating for the average price of a home where homes are greater than or equal to $350,000 (cached)`
FROM home_sales
WHERE price >= 350000 
GROUP BY view
ORDER BY view desc
''' 
spark.sql(query_cached).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+------------------------------------------------------------------------------------------------------+
|view|view rating for the average price of a home where homes are greater than or equal to $350,000 (cached)|
+----+------------------------------------------------------------------------------------------------------+
|  99|                                                                                            1061201.42|
|  98|                                                                                            1053739.33|
|  97|                                                                                            1129040.15|
|  96|                                                                                            1017815.92|
|  95|                                                                                             1054325.6|
|  94|                                                                                             1033536.2|
|  93|    

In [22]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partition")

In [23]:
# 11. Read the parquet formatted data.
partition_df=spark.read.parquet("home_sales_partition")

In [24]:
# 12. Create a temporary table for the parquet data.
partition_df.createOrReplaceTempView('home_sales_partition')

In [27]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()
query_parition= '''
SELECT view, round(avg(price),2) as `view rating for the average price of a home where homes are greater than or equal to $350,000 (parition)`
FROM home_sales_partition
WHERE price >= 350000 
GROUP BY view
ORDER BY view desc
''' 
spark.sql(query_parition).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------------------------------------------------------------------------------------------+
|view|view rating for the average price of a home where homes are greater than or equal to $350,000 (parition)|
+----+--------------------------------------------------------------------------------------------------------+
|  99|                                                                                              1061201.42|
|  98|                                                                                              1053739.33|
|  97|                                                                                              1129040.15|
|  96|                                                                                              1017815.92|
|  95|                                                                                               1054325.6|
|  94|                                                                                               103

In [28]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [29]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False